In [108]:
from pydantic import BaseModel
from typing import List

## Philosopher profile schema
class PhilosopherProfile(BaseModel):
    name: str
    school: str
    stance: str
    core_claims: List[str]
    argumentative_style: str
    primary_goal: str

# Schema for a set of philosophers debating a topic
class PhilosopherSet(BaseModel):
    topic: str
    opposing_topic: str
    philosophers: List[PhilosopherProfile]


In [109]:
from dotenv import load_dotenv
load_dotenv()
model = "openai/gpt-oss-20b"
# initialize the LLM with the model name and any desired parameters

from langchain_groq import ChatGroq

llm = ChatGroq(
    model=model,
    temperature=0.1,
    max_retries=2,
)

In [110]:
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict
import json
import re

## Define input state schema
class PhilosophyAgentState(TypedDict, total=False):
    topic: str
    philosopher_set: PhilosopherSet
    enriched_philosophers: list
    history: list
    final_dialogue: str
    turn_count: int

## philosopher Generator node
prompt = ChatPromptTemplate.from_template("""
You are a philosophy professor.

Given a philosophical concept, do the following:
1. Identify its strongest opposing philosophical position.
2. Create two philosopher profiles:
   - One defending the original concept
   - One defending the opposing concept

Use historical realism when possible.

Concept: {topic}

IMPORTANT: Respond ONLY with a valid JSON object. Do NOT use any tools. The JSON must have this exact structure:
{{
  "topic": "...",
  "opposing_topic": "...",
  "philosophers": [
    {{"name": "...", "school": "...", "stance": "...", "core_claims": [...], "argumentative_style": "...", "primary_goal": "..."}},
    {{"name": "...", "school": "...", "stance": "...", "core_claims": [...], "argumentative_style": "...", "primary_goal": "..."}}
  ]
}}
""")

def create_philosophers(state: PhilosophyAgentState) -> PhilosophyAgentState:
    topic = state.get("topic", "Free Will")
    # Use plain invoke to avoid tool-calling
    resp = llm.invoke(prompt.format(topic=topic))
    text = getattr(resp, 'content', None) or str(resp)
    
    # Parse JSON from response
    try:
        parsed = json.loads(text)
    except Exception:
        # Try to extract JSON object from text
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if not match:
            raise ValueError(f"Could not extract JSON from response: {text[:200]}")
        parsed = json.loads(match.group(0))
    
    # Validate and create PhilosopherSet
    philosopher_set = PhilosopherSet.parse_obj(parsed)
    return {**state, "philosopher_set": philosopher_set, "turn_count": 0}


In [111]:
from langgraph.graph import StateGraph
## Define the state schema for the graph

## Create the graph with proper state schema
graph_1 = StateGraph(PhilosophyAgentState)
graph_1.add_node("create_philosophers", create_philosophers)
graph_1.set_entry_point("create_philosophers")
graph_1.set_finish_point("create_philosophers")

philosopher_creator = graph_1.compile()


In [112]:
from tavily import TavilyClient
import os

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
if not TAVILY_API_KEY:
    raise RuntimeError('TAVILY_API_KEY not set in environment')

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def retrieve_philosophy_knowledge(philosopher):
    query = f"{philosopher.school} philosophy arguments criticisms"
    try:
        docs = tavily_client.search(query, max_results=5)
    except Exception as e:
        docs = []

    return {
        "philosopher": philosopher,
        "sources": docs
    }


In [113]:
def retrieval_map(state):
    philosophers = state["philosopher_set"].philosophers
    enriched = [
        retrieve_philosophy_knowledge(p) for p in philosophers
    ]
    return {**state, "enriched_philosophers": enriched, "history": []}


In [114]:
class DebateTurn(BaseModel):
    speaker: str
    argument: str
    question: str

def debate_turn(state):
    history = state.get("history", [])
    philosophers = state["enriched_philosophers"]
    turn_count = state.get("turn_count", 0)

    prompt_template = ChatPromptTemplate.from_template("""
You are {name}, a philosopher from the {school} tradition.

Your task:
- Respond directly to the previous philosopher's question or argument
- Defend your philosophical position
- Challenge the opponent's reasoning
- End with a probing philosophical question for them

Opponent's last argument:
{last_argument}

Your sources:
{sources}

IMPORTANT: Respond ONLY with a valid JSON object. Do NOT use any tools. The JSON must have this exact structure:
{{
  "speaker": "{name}",
  "argument": "...",
  "question": "..."
}}
""")

    turns = []

    # Each philosopher responds in sequence to the previous one's question
    for i, p in enumerate(philosophers):
        # Determine what the previous philosopher said
        if history:
            last_turn = history[-1]
            last_argument = last_turn.question if hasattr(last_turn, 'question') else last_turn.get('question', '')
        else:
            last_argument = f"Begin the debate on {state.get('topic', 'the topic')}"
        
        # Extract sources - Tavily returns a dict with 'results' key
        sources_list = p["sources"]
        if isinstance(sources_list, dict) and "results" in sources_list:
            sources_list = sources_list["results"]
        
        # Format sources safely
        if isinstance(sources_list, list):
            sources_text = "\n".join([f"- {s.get('title', s.get('query', 'Source'))}" for s in sources_list[:3]])
        else:
            sources_text = str(sources_list)[:500]
        
        # Use plain invoke to avoid tool-calling
        resp = llm.invoke(
            prompt_template.format(
                name=p["philosopher"].name,
                school=p["philosopher"].school,
                last_argument=last_argument,
                sources=sources_text
            )
        )
        text = getattr(resp, 'content', None) or str(resp)
        
        # Parse JSON response
        try:
            parsed = json.loads(text)
        except Exception:
            match = re.search(r'\{.*\}', text, re.DOTALL)
            if not match:
                parsed = {"speaker": p["philosopher"].name, "argument": text[:500], "question": "What do you think?"}
            else:
                parsed = json.loads(match.group(0))
        
        turn = DebateTurn.parse_obj(parsed)
        turns.append(turn)

    return {**state, "history": history + turns, "turn_count": turn_count + 1}


In [115]:

## node to format the final dialogue
def format_dialogue(state):
    dialogue = []
    for turn in state["history"]:
        # Handle both Pydantic objects and dicts
        speaker = turn.speaker if hasattr(turn, 'speaker') else turn.get('speaker', 'Unknown')
        argument = turn.argument if hasattr(turn, 'argument') else turn.get('argument', '')
        question = turn.question if hasattr(turn, 'question') else turn.get('question', '')
        
        dialogue.append(
            f"{speaker}:\n{argument}\n\nQuestion:\n{question}\n"
        )

    return {
        **state,
        "final_dialogue": "\n---\n".join(dialogue)
    }


In [116]:
from langgraph.graph import StateGraph, END

## Create the final graph
final_graph = StateGraph(PhilosophyAgentState)

final_graph.add_node("philosophers", create_philosophers)
final_graph.add_node("retrieve", retrieval_map)
final_graph.add_node("debate", debate_turn)
final_graph.add_node("format", format_dialogue)

final_graph.set_entry_point("philosophers")
final_graph.add_edge("philosophers", "retrieve")
final_graph.add_edge("retrieve", "debate")

# Loop debate 5 times, then go to format
def should_continue(state):
    turn_count = state.get("turn_count", 0)
    if turn_count < 5:
        return "debate"
    return "format"

final_graph.add_conditional_edges("debate", should_continue)
final_graph.add_edge("format", END)

philosophy_agent = final_graph.compile()


In [117]:
from IPython.display import display, Markdown

result = philosophy_agent.invoke({
    "topic": "Utilitarianism"
})

# print(result["final_dialogue"])
output = result["final_dialogue"]

display(Markdown(output))

C:\Users\Yusuf Solomon\AppData\Local\Temp\ipykernel_15132\4242937701.py:57: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  philosopher_set = PhilosopherSet.parse_obj(parsed)
C:\Users\Yusuf Solomon\AppData\Local\Temp\ipykernel_15132\2186775866.py:77: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  turn = DebateTurn.parse_obj(parsed)


John Stuart Mill:
The debate on utilitarianism must begin with a clear recognition that the moral law is the principle of the greatest happiness for the greatest number. Act utilitarianism, which evaluates each act solely on its immediate consequences, is vulnerable to the tyranny of the majority and to the neglect of moral duties that are essential for a just society. Rule utilitarianism, on the other hand, adopts rules that, if generally followed, tend to produce the greatest happiness. This approach preserves individual rights and promotes social stability while still aiming for the greatest overall well‑being. Critics often argue that utilitarianism reduces moral life to a mere calculation of pleasure and pain, but this criticism overlooks the qualitative distinctions between higher and lower pleasures that I have emphasized. Moreover, utilitarianism is not a blind pursuit of pleasure; it requires a careful assessment of the consequences of our actions, including the long‑term effects on society. Therefore, utilitarianism remains the most rational and humane ethical system, as it aligns moral deliberation with the ultimate aim of human flourishing.

Question:
If utilitarianism prioritizes the greatest happiness, how can it justify protecting the rights of a minority when doing so might reduce overall happiness?

---
Immanuel Kant:
I appreciate the invitation to discuss Utilitarianism, yet I must first point out that its foundational premise—maximizing overall happiness—fails to respect the intrinsic worth of each individual. Utilitarianism treats persons as instruments whose value is measured by the pleasure or utility they can produce, thereby violating the principle that humanity must always be treated as an end in itself. The categorical imperative demands that we act only according to maxims that can be universalized without contradiction; a maxim that allows one to sacrifice an innocent person for the greater good cannot be universalized, for it would erode the very moral law that protects all. Moreover, utilitarian calculations are inherently contingent on subjective valuations of pleasure and pain, which cannot be grounded in a rational, objective moral framework. Thus, while Utilitarianism may offer a seemingly pragmatic solution, it ultimately collapses under the weight of its own moral inadequacy. I challenge you to explain how a system that permits individuals to be treated merely as means can be justified as a legitimate moral theory.

Question:
How can you reconcile the imperative to treat each person as an end in themselves with a moral calculus that permits their instrumental use for the sake of aggregate happiness?

---
John Stuart Mill:
The Kantian injunction to treat each person as an end in themselves is not a categorical prohibition against instrumental use; rather, it is a normative guideline that, when interpreted as a rule, aligns perfectly with the utilitarian calculus. Rule utilitarianism teaches that we should adopt rules that, in general, maximize happiness. One such rule is the principle of respecting persons as ends, because a society that consistently treats individuals with dignity tends to produce the greatest aggregate welfare. The apparent conflict arises only if we read the Kantian imperative as a rigid, context‑free command that overrides all other considerations. In practice, utilitarianism allows for the instrumental use of individuals only when that use is justified by a net increase in happiness and does not violate the rule that, in the long run, respecting persons yields the greatest good. Thus, the two principles are reconciled by recognizing that the Kantian rule is itself a tool for achieving the utilitarian aim of the greatest happiness for the greatest number.

Question:
If we accept that a rule respecting persons can be part of a utilitarian framework, how do we determine which rules to adopt without risking a paradox where the rule itself becomes an instrument for maximizing happiness at the expense of the very ends it is meant to protect?

---
Immanuel Kant:
The imperative to treat each person as an end in themselves is not a mere preference but a universal moral law that follows from the very nature of rational agency. It is derived from the categorical imperative, which demands that we act only according to maxims that could be willed as universal laws. If we were to allow the instrumental use of a person for the sake of aggregate happiness, we would be prescribing a maxim that treats that person merely as a means to an end. Such a maxim cannot be universalized without contradiction, because it would require that all rational beings be treated as mere tools whenever it is convenient, thereby eroding the very dignity that makes them capable of moral law. The moral calculus of utilitarianism, which seeks to maximize overall happiness, is a heuristic that can guide decision‑making but it is not a moral law in itself. It cannot override the higher principle that each rational being must be respected as an end. Therefore, any attempt to reconcile the two is a logical inconsistency: the categorical imperative forbids the instrumental use of persons, and any moral calculus that permits it violates the very foundation of moral law. The opponent’s reasoning mistakenly treats the utilitarian calculus as a moral law, when in fact it is merely a descriptive tool that can be used to evaluate the consequences of actions that are already morally permissible.

Question:
If we cannot use any individual as a mere means, how then can we ever justify a policy that requires sacrificing a few people for the greater good of many?

---
John Stuart Mill:
Utilitarianism does not permit the use of any individual merely as a means; rather, it regards each person's happiness as a legitimate value that must be weighed in the calculation of the greatest good. The principle of utility requires that we consider the total happiness produced by an action, but it also demands that we respect the rights and dignity of individuals insofar as those rights contribute to overall well‑being. A policy that sacrifices a few for the greater good of many is permissible only if the net increase in happiness outweighs the loss, and only if the sacrifice does not violate fundamental rights that would ultimately undermine the aggregate good. Rule utilitarianism further refines this by endorsing rules—such as the rule against killing innocent people—that generally maximize happiness; only in exceptional circumstances would we break such a rule if doing so produced a greater overall benefit. Thus, utilitarianism can justify a policy that sacrifices a few for many, but only after a careful, transparent calculation that takes into account both the aggregate happiness and the moral rights of those individuals.

Question:
How can we ensure that our calculations of happiness are accurate enough to justify sacrificing individuals, and what safeguards can we put in place to prevent abuse of the principle of utility?

---
Immanuel Kant:
The claim that a policy which sacrifices a few for the greater good can be justified rests on a misunderstanding of the categorical imperative. The imperative demands that we treat humanity, in ourselves and in others, always as an end and never merely as a means. To sacrifice individuals for a supposed collective benefit is to use those individuals as instruments of an abstract calculation, thereby violating their inherent dignity. If we were to universalize such a maxim—‘it is permissible to sacrifice a few for the many’—the very notion of moral law would collapse, because the law would then allow the very practice it condemns. Moreover, the notion of ‘greater good’ is a teleological, consequentialist notion that presupposes that ends can be weighed against means, which is precisely what Kantian ethics rejects. Therefore, no policy that requires the deliberate sacrifice of individuals can be morally justified within a Kantian framework. The correct approach is to seek solutions that respect the autonomy and rationality of every person, even if that means accepting a lesser overall benefit.

Question:
If we cannot justify sacrificing individuals for the sake of a larger collective, how then should we morally evaluate situations where the well‑being of many is at stake but no individual is directly harmed?

---
John Stuart Mill:
The utilitarian principle does not hinge on the notion of sacrificing individuals; it is a calculus of the total sum of pleasures and pains. In a scenario where the well‑being of many is at stake but no single person is directly harmed, the moral evaluation proceeds by comparing the aggregate happiness that each possible action would produce. The opponent’s reasoning mistakenly conflates the idea of sacrifice with the utilitarian calculus. Even when no individual is harmed, the principle still demands that we choose the action that maximises the overall balance of pleasure over pain. Rule utilitarianism further refines this by endorsing rules that, on the whole, promote the greatest happiness while safeguarding individual rights. Thus, we can morally evaluate such situations by weighing the net increase in happiness, without invoking any unjust sacrifice. The challenge for the opponent is to recognize that utilitarianism is not a doctrine of sacrifice but a doctrine of maximisation, and that it can accommodate cases where no individual is harmed while still guiding us toward the most beneficial outcome for the collective.

Question:
If we accept that maximizing overall happiness is the moral yardstick, how do we determine when the happiness of a majority justifies the potential infringement of the rights of a minority, and what safeguards can we put in place to prevent the tyranny of the majority?

---
Immanuel Kant:
The moral worth of an action is not measured by the number of people who benefit or suffer, but by whether the maxim that guides the action can be willed as a universal law and whether it treats humanity—both the actor and the recipients—as ends in themselves. In a situation where no individual is directly harmed, the opponent’s intuition that the action is automatically permissible is a mistake of consequentialist thinking. Even absent direct harm, an act may still violate duty if it uses people merely as a means to an end, or if it undermines their autonomy or dignity. For example, a policy that maximizes collective well‑being by coercing individuals into a particular behavior, without their consent, cannot be universalized without contradiction, because it would require that all persons be treated as instruments. Thus, Kantian ethics demands a careful examination of the maxim’s universalizability and its respect for persons, regardless of the presence or absence of direct harm. The moral evaluation must therefore focus on duty and respect for humanity, not on aggregate outcomes.

Question:
If an action does not directly harm any individual but results in the loss of autonomy or dignity for a group, can we still consider it morally permissible under Kantian ethics, or does the categorical imperative require us to act otherwise?

---
John Stuart Mill:
The utilitarian calculus is not limited to the absence of direct harm; it requires us to weigh all consequences, including the loss of autonomy or dignity for a group. Such a loss diminishes the overall happiness of the members of that group and, by extension, the society that values their well‑being. Therefore, an action that erodes group autonomy cannot be morally permissible if it reduces the greatest happiness. Kant’s categorical imperative, while demanding that we treat individuals as ends in themselves, does not exempt us from considering the dignity of groups. If an action treats a group merely as a means to an end—by sacrificing their autonomy for a supposed greater good—it violates the imperative’s requirement that we never use any person or collective merely as a tool. Thus, Kantian ethics would also require us to act otherwise, not because it merely looks for direct harm but because it respects the intrinsic worth of all persons, whether individual or collective. The utilitarian stance, however, provides a clearer, more flexible framework for assessing the net happiness lost when autonomy is stripped away, and it compels us to seek alternatives that preserve both individual and group dignity while maximizing overall well‑being.

Question:
If a society values collective autonomy above individual happiness, how should we balance the utilitarian imperative to maximize overall happiness with the Kantian duty to respect the intrinsic worth of each person?

---
Immanuel Kant:
Under the first formulation of the categorical imperative, we may act only on maxims that can be universalized without contradiction. A maxim that permits the loss of autonomy or dignity for a group cannot be universalized, because if every rational agent were to adopt such a maxim, the very concept of autonomy that underpins rational agency would be undermined. The second formulation demands that we treat humanity, whether in ourselves or others, always as an end and never merely as a means. By allowing an action that erodes the autonomy or dignity of a group, we are using that group as a mere instrument to achieve some end, thereby violating the imperative. The opponent’s reasoning mistakenly treats group autonomy as a secondary concern; however, autonomy and dignity are not merely individual rights but are the very conditions that make rational agency possible for all members of a community. Consequently, any action that results in the loss of group autonomy or dignity is morally impermissible under Kantian ethics.

Question:
If we accept that the loss of group autonomy is impermissible, how should we evaluate actions that preserve individual autonomy but potentially diminish the collective autonomy of a community?
